In [26]:
import pandas as pd 
import solid 
import json
from urllib import urlencode
from subprocess import call

from solid.utils import *
extrude_along_path

<function solid.utils.extrude_along_path>

In [27]:
query = {'q':'''
        SELECT cartodb_id,  
        ST_AsGeoJSON( ST_SIMPLIFY(the_geom,0.01)),
        percent_white from 
        us_census_acs2013_5yr_county_copy 
        where ST_IsEmpty( ST_Simplify(the_geom,0.01) ) = false ''', 'format':'csv'}

In [28]:
data = pd.read_csv('https://observatory.cartodb.com/api/v2/sql?'+urlencode(query))

In [29]:
def extract_geometry(data):
    path = json.loads(data.st_asgeojson)['coordinates'][0][0]
    
    return render()((linear_extrude(height=data.percent_white/10.0)(polygon(points=path))))

In [30]:
polygons = data.apply(extract_geometry, axis=1)

In [33]:
model = union()(list(polygons.as_matrix()))

In [ ]:
with open('us_model2.scad', 'w') as f:
    f.write(scad_render(model).replace('triangles','faces'))
print call(['/Applications/OpenSCAD.app/Contents/MacOS/OpenSCAD',
          '-o', 
          'us_model2.stl', 
          'us_model2.scad'])   